In [ ]:
from tensorflow.keras.datasets import cifar10

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.rcParams['figure.figsize'] = (10, 8)

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
len(x_train), len(x_test)

In [ ]:
x_train[0].shape

In [ ]:
test_image = x_train[1]
test_image = test_image / 255
show_image(test_image)

In [ ]:
def show_image(img):
    plt.imshow(img)
 
show_image(x_train[1])

In [ ]:
from PIL import Image
test_image = Image.open('test_img.jpg')
test_image = np.array(test_image)
test_image = test_image / 255
show_image(test_image)

In [ ]:
import numpy as np

def apply_gaussian_noise(img, simga):
    noise = np.random.normal(0, simga, img.shape)
    return np.clip(img + noise, 0, 1)

def apply_salt_and_pepper_noise(img, prob):
    s_and_p = np.random.rand(*img.shape)
    new_img = img.copy()
    new_img[s_and_p < prob] = 0
    new_img[s_and_p > 1 - prob] = 1
    return new_img

In [ ]:
show_image(apply_gaussian_noise(test_image, 0.07))

In [ ]:
show_image(apply_salt_and_pepper_noise(test_image, 0.1))

In [ ]:
show_image(test_image)

In [ ]:
np.random.normal(0, 0.05, (3, 3, 1))

In [ ]:
import skimage

In [ ]:
noise = skimage.util.random_noise(test_image, var=0.25**2)
show_image(noise)

In [ ]:
noise = skimage.util.random_noise(test_image, mode='poisson')
show_image(noise)

In [ ]:
noise = skimage.util.random_noise(test_image, mode='s&p')
show_image(noise)

In [ ]:
x = np.concatenate((x_train, x_test), axis=0)
x = x / 255
x.shape

In [ ]:
def generate_images_with_noise(y, modes, mode_partition=None):
    if mode_partition:
        if len(mode_partition) != len(modes):
            raise ValueError('Number of partitions should match number of modes')
        if sum(mode_partition) > len(y):
            raise ValueError('Total partitions ' + sum(mode_partition) + ' > number of images ' + len(y))
    else:
        mode_partition = [len(y) // len(modes)] * len(modes)
        
    x = []
    for partition in mode_partition:
        i = 0 
        for image in y[i:partition+i]:
            x.append(skimage.util.random_noise(test_image, mode='s&p'))
        i += partitio
    

In [ ]:
def load_data():
    (x_train, _), (x_test, _) = cifar10.load_data()
    y = np.concatenate((x_train, x_test), axis=0)
    return y

def preprocess_data(x):
    x = x / 255
    return x

def generate_images_with_gaussian_noise(y, sigma=0.1):
    x = []
    for img in y:
        x.append(skimage.util.random_noise(img, var=sigma**2, mode='gaussian'))
    return np.array(x)

def generate_images_with_poisson_noise(y):
    x = []
    for img in y:
        x.append(skimage.util.random_noise(img, mode='poisson'))
    return np.array(x)

def generate_images_with_s_and_p_noise(y, amount=0.05, salt_vs_pepper=0.5):
    x = []
    for img in y:
        x.append(skimage.util.random_noise(img, mode='s&p', amount=amount,
                                           salt_vs_pepper=salt_vs_pepper))
    return np.array(x)

In [ ]:
y = load_data()

In [ ]:
y = preprocess_data(y)

In [ ]:
x = generate_images_with_gaussian_noise(y)

In [ ]:
x.shape

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, UpSampling2D, Dropout, Input, Flatten

class GAN:
    def __init__(self, x, y):
        self.input_shape = x[0].shape
        self.x = x
        self.y = y
        
        self.generator = self._get_generator()
        self.discriminator = self._get_discriminator()
        self.gan = self._get_gan(self.generator, self.discriminator)
        
    def _get_generator(self):
        generator = Sequential()
        generator.add(Conv2D(filters=32, kernel_size=(3, 3),
                             input_shape=self.input_shape,
                             activation='relu', padding='same'))
        #generator.add(Dropout(0.25))
        #generator.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
        #generator.add(Conv2D(filters=32, kernel_size=(3, 3),
        #                     activation='relu', padding='same'))
        #generator.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
        
        generator.add(Conv2D(filters=32, kernel_size=(3, 3),
                             activation='relu', padding='same'))
        #generator.add(UpSampling2D((2, 2)))
        #generator.add(Conv2D(filters=32, kernel_size=(3, 3),
        #                     activation='relu', padding='same'))
        #generator.add(UpSampling2D((2, 2)))
        generator.add(Conv2D(filters=3, kernel_size=(3, 3),
                             activation='sigmoid', padding='same'))
        generator.compile(loss='binary_crossentropy', optimizer='adadelta')
        return generator
    
    def _get_discriminator(self):
        discriminator = Sequential()
        discriminator.add(Conv2D(filters=32, kernel_size=(3, 3),
                             input_shape=self.input_shape,
                             activation='relu'))
        discriminator.add(Conv2D(filters=32, kernel_size=(3, 3),
                             activation='relu'))
        discriminator.add(MaxPooling2D(pool_size=(2, 2)))
        discriminator.add(Conv2D(filters=32, kernel_size=(3, 3),
                             activation='relu'))
        discriminator.add(MaxPooling2D(pool_size=(2, 2)))
        discriminator.add(Dropout(0.25))
        discriminator.add(Flatten())
        discriminator.add(Dense(512, activation='relu'))
        discriminator.add(Dropout(0.5))
        discriminator.add(Dense(1, activation='sigmoid'))
        discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

        return discriminator
        
    def _get_gan(self, generator, discriminator):
        discriminator.trainable = False
        
        gan_input = Input(shape=self.input_shape)
        x = generator(gan_input)
        gan_output = discriminator(x)
        gan = Model(inputs=gan_input, outputs=gan_output)
        gan.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        return gan
    
    def train(self, epochs, batch_size=128):
        accuracy = 0.5
        
        for e in range(1, epochs+1):
            print('-'*10, 'Epoch %s' % e, '-'*10)
            
            batch = np.random.randint(0, self.x.shape[0], size=batch_size)
            image_noise_batch = self.x[batch]
            image_batch = self.y[batch]
            
            generated = self.generator.predict(image_noise_batch)
            X = np.concatenate([image_batch, generated])
            
            y_dis = np.zeros(2*batch_size)
            y_dis[:batch_size] = 1
            
            self.discriminator.trainable = True
            self.generator.trainable = False
            
            accuracy = 0.5
            while accuracy < 0.9:
                disc_loss, accuracy = self.discriminator.train_on_batch(X, y_dis)
                print('Discriminator accuracy:', accuracy)
                
            y_gen = np.ones(batch_size)
            self.discriminator.trainable = False
            self.generator.trainable = True
            gan_accuracy = 0
            while gan_accuracy < 0.6:
                gan_loss, gan_accuracy = self.gan.train_on_batch(image_noise_batch, y_gen)
                print('GAN accuracy:', gan_accuracy)
            
            print('Discriminator loss:', disc_loss, 
                  'Discriminator accuracy:', accuracy,
                  'GAN loss:', gan_loss,
                  'GAN accuracy:', gan_accuracy)
            
            if e == 1 or e % 5 == 0:
                self.plot_images(e)
            
    def plot_images(self, epoch):
        image_noise_batch = self.x[np.random.randint(0, self.x.shape[0], size=2)]
        generated_images = self.generator.predict(image_noise_batch)
        
        fig, ax = plt.subplots(nrows=2, ncols=2)
        
        ax[0][0].imshow(image_noise_batch[0])
        ax[0][1].imshow(generated_images[0])
        ax[1][0].imshow(image_noise_batch[1])
        ax[1][1].imshow(generated_images[1])
        plt.show()
        

In [ ]:
gan = GAN(x, y)

In [ ]:
gan.train(epochs=200)

In [ ]:
show_image(y[1])

In [ ]:
#Сделать grayscale